In [1]:
import numpy as np
import networkx as nx
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number
W=10000

In [3]:
file_name='full_graph_{}_{}.txt'.format(n,N)
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)

# Reading graph

In [4]:
con=[[] for _ in range(n+1)]
edges=set()

In [5]:
with open('test/'+file_name, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        
        if x>N and y>N:
            
            con[x-N].append((y-N,w))
            con[y-N].append((x-N,w))
            edges.add((x-N,y-N,w))

           
    f.close()

In [6]:
#con
edges

{(1, 3, 0.988270793001535),
 (1, 5, 0.08892011580397619),
 (3, 1, 0.988270793001535),
 (3, 4, 0.7447419193996232),
 (4, 3, 0.7447419193996232),
 (5, 1, 0.08892011580397619)}

# Reading shortest paths

In [7]:
sh_path=[[] for _ in range(N+1)]

for i in range(n+1):
    sh_path[i]=[[] for _ in range(n+1)]

In [8]:
with open('test/'+file_name_shortest_path, 'r') as f:
    
    n=int(f.readline())
    
    for i in range(1,n+N+1):
        for j in range(1,n+N+1):
            if i!=j and :
                x,y=map(int,f.readline().split())
                #print('x,y',x,y)
                
                while True:
                    line1=f.readline()
                    #print('line1',line1)
                    
                    if not line1.strip():
                        break

                    line2=f.readline()
                    
                    w=float(line1)
                    path=list(map(int,line2.split()))
                    #print('path',path)
                    #print('w',w)

                    sh_path[i][j].append((w,path))


ValueError: invalid literal for int() with base 10: '5 10\n'

In [ ]:
#sh_path[1][3][2]

# Figuring out the coefficients alpha

In [ ]:
alpha=set()

In [ ]:
def belong_to_path():
   
    for i in range(n+1):
        for j in range(n+1):
            if i!=j:
                for s in range(len(sh_path[i][j])):
                    path=sh_path[i][j][s][1]
                    l=len(path)
                    
                    for k in range(l-1):
                        alpha.add((path[k],path[k+1],s,i,j))

In [ ]:
belong_to_path()

In [ ]:
alpha

# Collecting terms

In [ ]:
def mult(a):
    d={}
    
    for q in a:
        for e in q['term']:
            for e2 in q['term']:
                
                x,y=e
                z,t=e2
                
                if x<z:
                    x,z=z,x
                    y,t=t,y
                
                coeff=q['coeff']
                            
                if x!='C' and z!='C':
                    d[(x,z)]=d[(x,z)]+coeff*y*t if (x,z) in d else coeff*y*t
                    
                if x=='C' and z!='C':
                    d[(z,z)]=d[(z,z)]+coeff*y*t if (z,z) in d else coeff*y*t
                    
                if x!='C' and z=='C':
                    d[(x,x)]=d[(x,x)]+coeff*y*t if (x,x) in d else coeff*y*t
                    
        #print(d)
        

                    
    return d

# Example

In [ ]:
form=[{'coeff':1, 'term':[('y2',1)]}, {'coeff':4, 'term':[('x1',1), ('x2',1), ('y2',1)]}, {'coeff':1, 'term':[('x2',1)]}, {'coeff':1, 'term':[('y1',1)]}, 
      {'coeff':1, 'term':[('x1',1), ('y2',1)]}, {'coeff':9, 'term':[('x1',1), ('y1',1)]}, {'coeff':25, 'term':[('x1',1), ('x2',1), ('C',-1)]}, {'coeff':25, 'term':[('y1',1), ('y2',1), ('C',-1)]}]

In [ ]:
Q=mult(form) #we ignore a constant term

In [ ]:
Q #we are ignoring the constant term and using the fact that x^2=x, x in {0,1}

# Real network example

In [ ]:
form=[0]*(n+1)

for i in range(n+1):
    form[i]=[0]*(n+1)

In [ ]:
D=np.random.rand(n+1,n+1)

In [ ]:
form=[]
xs=set()

In [ ]:
for e in edges:
    x,y,w=e
    term=[]
    coeff=0
    
    for i in range(n+1):
        for j in range(n+1):
            if i!=j:
                
                for s in range(len(sh_path[i][j])):
                    
                    if (x,y,s,i,j) in alpha:
                        coeff=1.0/w
                        name='x'+'_'+str(s)+'_'+str(i)+'_'+str(j)
                        term.append((name,D[i][j]))
                        xs.add(name)
            
            
                if coeff!=0:
                    form.append({'coeff':coeff, 'term':term})                

In [ ]:
len(xs)

In [ ]:
edges

In [ ]:
xs

In [ ]:
for i in range(n+1):
        for j in range(n+1):
            if i!=j:
    
                term=[]

                for s in range(len(sh_path[i][j])):
                    name='x'+'_'+str(s)+'_'+str(i)+'_'+str(j)

                    if name in xs:
                        term.append((name,1))


                term.append(('C',-1))
                
                #print('term',term)
                
                if coeff!=0:
                    form.append({'coeff':W, 'term':term})   
                        
                    
                    

In [ ]:
form

In [ ]:
Q=mult(form)

In [ ]:
Q

In [ ]:
len(Q)

In [ ]:
sampler = EmbeddingComposite(DWaveSampler())

In [ ]:
response=sampler.sample_qubo(Q, num_reads=1)

In [ ]:
for data in response.data(['sample','energy', 'num_occurrences']):
    
    print(data.sample, 'energy', data.energy, 'num_occurrences', data.num_occurrences) 